# North coast kelp data conversion

Since Laura only shared with me an MS Access DB file (.mdb), I'll have to start by extracting the data from there.

Resources:
- [Wiki for pyodbc](https://github.com/mkleehammer/pyodbc/wiki)

In [1]:
## Imports

import pandas as pd
import numpy as np
import csv, pyodbc

## Connect to db and retrieve data

Note that (as described in the Wiki) the original database filename did not work because it included underscores. I renamed 'Abalone_DiveSurveys_EH_06242020.mdb' to 'AbaloneDiveSurveys-06242020.mdb' to fix this problem

In [6]:
## Check available access driver

[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

In [33]:
## Connect to db

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\dianalg\PycharmProjects\PythonScripts\MPA data integration\North coast kelp\AbaloneDiveSurveys-06242020.mdb;'
    )
cnxn = pyodbc.connect(conn_str)
crsr = cnxn.cursor()

# Print all table names
for table_info in crsr.tables(tableType='TABLE'):
    print(table_info.table_name)

Paste Errors
Switchboard Items
Tbl_Diver_Lookup
Tbl_Format_Lookup
Tbl_New_size
Tbl_SppSeen
Tbl_Survey_Lookup
tblCounts
tblHabitat
tblNewGrowth
tblQuadrat
tblSite
TblSpecies
tblSubstrate
tblSurvey
tblSwimGroups


In [35]:
cols = []
for row in crsr.columns(table='tblSurvey'):
    print(row.column_name)
    cols.append(row.column_name)
cols

Survey_ID
SiteID
Survey_Num
SURVEY


UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 32-33: illegal encoding

In [34]:
[row.column_name for row in crsr.columns(table='tblSurvey')]
    

UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 32-33: illegal encoding

In [30]:
## Get data and close connection

SQL = 'SELECT * FROM tblSurvey;'
rows = crsr.execute(SQL).fetchall()
cols = [row.column_name for row in crsr.columns(table='tblSurvey')]

crsr.close()
cnxn.close()

UnicodeDecodeError: 'utf-16-le' codec can't decode bytes in position 32-33: illegal encoding

In [36]:
crsr.close()
cnxn.close()

In [15]:
## Write data to .csv file

with open('tblSurvey.csv', 'w') as fou:
    csv_writer = csv.writer(fou) # default field-delimiter is ","
    csv_writer.writerows(rows)

## Load data

In [19]:
## Load data

pd.read_csv('tblSurvey.csv', header=None, encoding='ANSI')

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,4790,FR,FR18-D5-1,Transect - 30m (Rapid Emergent),2018-07-12 00:00:00,NaN,NaN,29,K. SOWUL,NaN,...,0.0,0.0,0.0,NaN,NaN,RAPID EMERGENT SURVEY. D5: 1 OUT OF 2 TRANSECT...,NaN,NaN,NaN,NaN
1,4919,ALB,ALB18-A02-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,NaN,5,B. BENNETT,NaN,...,0.0,0.0,0.0,NaN,NaN,RAPID EMERGENT SITE A02: 1 OF 2 TRANSECTS. HEA...,NaN,NaN,NaN,NaN
2,4920,ALB,ALB18-A02-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,NaN,6,B. BENNETT,NaN,...,0.0,0.0,0.0,NaN,NaN,RAPID EMERGENT SITE A02: 2 OF 2 TRANSECTS. HEA...,NaN,NaN,NaN,NaN
3,4915,ALB,ALB18-A1-1,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,NaN,1,M. MCPHERSON,NaN,...,0.0,0.0,0.0,NaN,NaN,RAPID EMERGENT SITE A1: 1 OF 4 TRANSECTS. HEAD...,NaN,NaN,NaN,NaN
4,4916,ALB,ALB18-A1-2,Transect - 30m (Rapid Emergent),2018-08-30 00:00:00,NaN,NaN,2,C. CATTON,NaN,...,0.0,0.0,0.0,NaN,NaN,RAPID EMERGENT SITE A1: 2 OF 4 TRANSECTS. HEAD...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,4949,YB,YB19-2-4,ARM,2019-09-16 00:00:00,NaN,NaN,8,I. TANIGUCHI,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3900,4950,YB,YB19-3-1,ARM,2019-09-16 00:00:00,NaN,NaN,9,L. ROGERS-BENNETT,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3901,4951,YB,YB19-3-2,ARM,2019-09-16 00:00:00,NaN,NaN,10,S. KAWANA,NaN,...,0.0,0.0,0.0,NaN,NaN,WARTY SEA CUCUMBER SIZES (MM): 25X54X50).,NaN,NaN,NaN,NaN
3902,4952,YB,YB19-3-3,ARM,2019-09-16 00:00:00,NaN,NaN,11,I. TANIGUCHI,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## This is really messy and came with no headers